# Heart Disease Prediction and EDA

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/personal-key-indicators-of-heart-disease/2020/heart_2020_cleaned.csv
/kaggle/input/personal-key-indicators-of-heart-disease/2022/heart_2022_with_nans.csv
/kaggle/input/personal-key-indicators-of-heart-disease/2022/heart_2022_no_nans.csv


In [2]:
import pandas as pd

# Load the dataset
file_path = '/kaggle/input/personal-key-indicators-of-heart-disease/2022/heart_2022_no_nans.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())

     State     Sex GeneralHealth  PhysicalHealthDays  MentalHealthDays  \
0  Alabama  Female     Very good                 4.0               0.0   
1  Alabama    Male     Very good                 0.0               0.0   
2  Alabama    Male     Very good                 0.0               0.0   
3  Alabama  Female          Fair                 5.0               0.0   
4  Alabama  Female          Good                 3.0              15.0   

                                     LastCheckupTime PhysicalActivities  \
0  Within past year (anytime less than 12 months ...                Yes   
1  Within past year (anytime less than 12 months ...                Yes   
2  Within past year (anytime less than 12 months ...                 No   
3  Within past year (anytime less than 12 months ...                Yes   
4  Within past year (anytime less than 12 months ...                Yes   

   SleepHours            RemovedTeeth HadHeartAttack  ... HeightInMeters  \
0         9.0            Non

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246022 entries, 0 to 246021
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      246022 non-null  object 
 1   Sex                        246022 non-null  object 
 2   GeneralHealth              246022 non-null  object 
 3   PhysicalHealthDays         246022 non-null  float64
 4   MentalHealthDays           246022 non-null  float64
 5   LastCheckupTime            246022 non-null  object 
 6   PhysicalActivities         246022 non-null  object 
 7   SleepHours                 246022 non-null  float64
 8   RemovedTeeth               246022 non-null  object 
 9   HadHeartAttack             246022 non-null  object 
 10  HadAngina                  246022 non-null  object 
 11  HadStroke                  246022 non-null  object 
 12  HadAsthma                  246022 non-null  object 
 13  HadSkinCancer              24

### Display unique values for each column

In [4]:
# Iterate through each column and process based on data type
for column in df.columns:
    if df[column].dtype == 'object':  # Categorical variables
        unique_values = df[column].unique()
        print(f"Column: {column}")
        print(f"Unique Values: {unique_values}\n")
    else:  # Numeric variables
        col_min = df[column].min()
        col_max = df[column].max()
        print(f"Column: {column}")
        print(f"Range: {col_min} to {col_max}\n")

Column: State
Unique Values: ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming' 'Guam' 'Puerto Rico'
 'Virgin Islands']

Column: Sex
Unique Values: ['Female' 'Male']

Column: GeneralHealth
Unique Values: ['Very good' 'Fair' 'Good' 'Excellent' 'Poor']

Column: PhysicalHealthDays
Range: 0.0 to 30.0

Column: MentalHealthDays
Range: 0.0 to 30.0

Column: LastCheckupTime
Unique Values: ['Within past year (anytime less than 12 months ago)'
 '5 or mor

Here’s a brief rundown of what each column in the dataset represents:

`State` - State FIPS Code\
`Sex` - Sex of Respondent\
`GeneralHealth` - Would you say that in general your health is:\
`PhysicalHealthDays` - Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?\
`MentalHealthDays` - Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?\
`LastCheckupTime` - About how long has it been since you last visited a doctor for a routine checkup?\
`PhysicalActivities` - During the past month, other than your regular job, did you participate in any physical activities or exercises such as running, calisthenics, golf, gardening, or walking for exercise?\
`SleepHours` - On average, how many hours of sleep do you get in a 24-hour period?\
`RemovedTeeth` - Not including teeth lost for injury or orthodontics, how many of your permanent teeth have been removed because of tooth decay or gum disease?\
`HadHeartAttack` - (Ever told) you had a heart attack, also called a myocardial infarction?\
`HadAngina` - (Ever told) (you had) angina or coronary heart disease?\
`HadStroke` - (Ever told) (you had) a stroke.\
`HadAsthma` - (Ever told) (you had) asthma?\
`HadSkinCancer` - (Ever told) (you had) skin cancer that is not melanoma?\
`HadCOPD` - (Ever told) (you had) C.O.P.D. (chronic obstructive pulmonary disease), emphysema or chronic bronchitis?\
`HadDepressiveDisorder` - (Ever told) (you had) a depressive disorder (including depression, major depression, dysthymia, or minor depression)?\
`HadKidneyDisease` - Not including kidney stones, bladder infection or incontinence, were you ever told you had kidney disease?\
`HadArthritis` - (Ever told) (you had) some form of arthritis, rheumatoid arthritis, gout, lupus, or fibromyalgia?  (Arthritis diagnoses include: rheumatism, polymyalgia rheumatica; osteoarthritis (not osteporosis); tendonitis, bursitis, bunion, tennis elbow; carpal tunnel syndrome, tarsal tunnel syndrome; joint infection, etc.)\
`HadDiabetes` - (Ever told) (you had) diabetes?\
`DeafOrHardOfHearing` - Are you deaf or do you have serious difficulty hearing?\
`BlindOrVisionDifficulty` - Are you blind or do you have serious difficulty seeing, even when wearing glasses?\
`DifficultyConcentrating` - Because of a physical, mental, or emotional condition, do you have serious difficulty concentrating, remembering, or making decisions?\
`DifficultyWalking` - Do you have serious difficulty walking or climbing stairs?\
`DifficultyDressingBathing` - Do you have difficulty dressing or bathing?\
`DifficultyErrands` - Because of a physical, mental, or emotional condition, do you have difficulty doing errands alone such as visiting a doctor´s office or shopping?\
`SmokerStatus` - Four-level smoker status:  Everyday smoker, Someday smoker, Former smoker, Non-smoker\
`ECigaretteUsage` - Would you say you have never used e-cigarettes or other electronic vaping products in your entire life or now use them every day, use them some days, or used them in the past but do not currently use them at all?\
`ChestScan` - Have you ever had a CT or CAT scan of your chest area?\
`RaceEthnicityCategory` - Five-level race/ethnicity category\
`AgeCategory` - Fourteen-level age category\
`HeightInMeters` - Reported height in meters\
`WeightInKilograms` - Reported weight in kilograms\
`BMI` - Body Mass Index (BMI)\
`AlchoholDrinkers` - Adults who reported having had at least one drink of alcohol in the past 30 days.\
`HIVTesting` - Adults who have ever been tested for HIV\
`FluVaxLast12` - During the past 12 months, have you had either flu vaccine that was sprayed in your nose or flu shot injected into your arm?\
`PneumoVaxEver` - Have you ever had a pneumonia shot also known as a pneumococcal vaccine?\
`TetanusLast10Tdap` - Have you received a tetanus shot in the past 10 years? Was this Tdap, the tetanus shot that also has pertussis or whooping cough vaccine?\
`HighRiskLastYear` - You have injected any drug other than those prescribed for you in the past year. You have been treated for a sexually transmitted disease or STD in the past year. You have given or received money or drugs in exchange for sex in the past year.\
`CovidPos` - Has a doctor, nurse, or other health professional ever told you that you tested positive for COVID 19?
